In [8]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = 'gcp-starter'

In [40]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [7]:
# Extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()

    return documents

In [10]:
extracted_data = load_pdf("data/")

In [12]:
#extracted_data

In [13]:
# Create text chunks
def text_splitter(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [14]:
text_chunks = text_splitter(extracted_data)

In [15]:
len(text_chunks)

7020

In [16]:
# Download embedding model
def download_hugging_face_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
embeddings = download_hugging_face_model()

In [18]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [19]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [21]:
# query_result

In [30]:
index_name = "medical-chatbot"
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name = index_name)

In [32]:
query = "What are allergies?"
docs = docsearch.similarity_search(query, k=3)
print(docs[0].page_content)

GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE


In [33]:
prompt_template = """
Use the following piece of information and answer the user's questions
If you don't know the answer, just say that you don't know the answer, don't try to make things up
Context: {context}
Question: {question}

Only return the helpful answer below and nothing else
Helpful answer:
"""

In [35]:
prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
chain_type_kwargs={"prompt":prompt}

In [41]:
llm = CTransformers(model='model/llama-2-7b-chat.ggmlv3.q4_0.bin',
                    model_type="llama",
                    config={'max_new_tokens': 512,
                            'temperature': 0.8})

AttributeError: module 'langchain' has no attribute 'verbose'

In [42]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever= docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents= True,
    chain_type_kwargs=chain_type_kwargs
)

NameError: name 'llm' is not defined

In [ ]:
while True:
    user_input = input("Input Prompt: ")
    result = qa({"query":user_input})
    print("Response: ", result["result"])